In [1]:
## Exercise 3.4: Let n = 507527 and e = 220007 be the public integers in an RSA key. Using Wiener's method:

# 1. find the private key d.
# 2. find the factors of n.

In [61]:
import math
import random as rand

n = 507527
e = 220007

In [22]:
def phi(x):
    """
    Euler's totient function. The output is all numbers between 1 and x - 1 that are coprime to x.
    """
    results = []
    
    for i in range(x):
        if (math.gcd(i,x) == 1):
            results.append(i)
            
    return results

In [23]:
# 1. find the private key d:

# First, we need to find the continued fraction expansion of alpha = e/n

alpha = e/n
print('alpha = e/n = ', alpha)

alpha = e/n =  0.4334882676192597


In [24]:
alpha_all = [0]*10
a_all = [0]*len(alpha_all)

alpha_all[0] = alpha

for i in range(len(alpha_all) - 1):
    a_all[i] = int(alpha_all[i])
    alpha_all[i+1] = 1/(alpha_all[i] - a_all[i])
    
print('All alpha´s:', alpha_all)
print('All a´s:', a_all)

All alpha´s: [0.4334882676192597, 2.306867508761085, 3.2587353546724342, 3.8649530570185364, 1.1561321066913917, 6.404832556167225, 2.470157068066749, 2.1269487750379374, 7.877192983557025, 1.1399999985692901]
All a´s: [0, 2, 3, 3, 1, 6, 2, 2, 7, 0]


In [25]:
# Next step is to find all the convergents u_i/v_i for i >= 1 of alpha:

u_i = [0]*len(alpha_all)
v_i = [0]*len(alpha_all)

u_i[0] = 1
u_i[1] = a_all[0]

v_i[0] = 0
v_i[1] = 1

for i in range(2,len(alpha_all)):
    u_i[i] = a_all[i-1]*u_i[i-1] + u_i[i-2]
    v_i[i] = a_all[i-1]*v_i[i-1] + v_i[i-2]

print('u_i:', u_i)
print('v_i:', v_i)
print('\n')

print('The fractions clearly converge to e/n:')
for i in range(1,len(alpha_all)):
    print('{0} / {1} = {2}'.format(u_i[i], v_i[i], u_i[i]/v_i[i]))

u_i: [1, 0, 1, 3, 10, 13, 88, 189, 466, 3451]
v_i: [0, 1, 2, 7, 23, 30, 203, 436, 1075, 7961]


The fractions clearly converge to e/n:
0 / 1 = 0.0
1 / 2 = 0.5
3 / 7 = 0.42857142857142855
10 / 23 = 0.43478260869565216
13 / 30 = 0.43333333333333335
88 / 203 = 0.43349753694581283
189 / 436 = 0.4334862385321101
466 / 1075 = 0.43348837209302327
3451 / 7961 = 0.43348825524431606


In [56]:
# Next step is to check whether the private key d corresponds to one of the v_i's. In order to check that, we choose
# some message m:

def find_d(m, alphas, v_i, e, n):
    for i in range(len(alphas)):
        current = pow(m, e*v_i[i], n)
        if (current == m):
            return v_i[i]

m = 100
d = find_d(m,alpha_all, v_i, e, n)

# Check that this value of d is actually correct
n_phi = phi(n)
assert(pow(e*d, 1, len(n_phi)) == 1)

print('The private key d is:', d)

The private key d is: 23


In [67]:
# 2. find the factors of n:

# find s,r such that ed - 1 = 2^s * r, where r is odd:

def find_s_and_r(e, d):
    """
    find s and r such that e*d - 1 = 2^s * r, where r is odd.
    """
    ed_minus1 = e*d - 1
    r = ed_minus1
    s = 1
    
    while((r % 2) == 0):
        s += 1
        r //= 2
        print(r)
        
    s -= 1
        
    return s, r

s, r = find_s_and_r(e, d)

assert (2**s * r == ed_minus1)
print('\n')
print('s = {0}, r = {1}, 2^s * r = {2} = ed_minus1'.format(s, r, 2**s * r))

2530080
1265040
632520
316260
158130
79065


s = 6, r = 79065, 2^s * r = 5060160 = ed_minus1


In [68]:
# choose a random value w, 1 <= w < n:

w = rand.randint(1,n-1)
print('w = ', w)

w =  84709


In [69]:
# Find p and q such that p * q = n

def factor_n(w, r, n):
    """
    Factoring an RSA modulus - if d is known.
    """
    if (1 < math.gcd(n, w) < n):
        p = w
        q = n // p
        return 'p: {0}, q: {1}, p*q = {2} = n'.format(p, q, p*q)
    else:
        v = pow(w, r, n)
        
        if (v == 1):
            return 'failure'
        
        while (v != 1):
            v_0 = v
            v = pow(v, 2, n)
            
        if ((v_0 == -1) % n):
            return 'failure'
        else:
            p = math.gcd(v_0 + 1, n)
            q = n // p
            return 'p: {0}, q: {1}, p*q = {2} = n'.format(p, q, p*q)

        
print(factor_n(w, r, n))
assert (p*q == n)

p: 1009, q: 503, p*q = 507527 = n
